## Proyecto análisis de datos 

En este proyecto se quiere intentar entender la problematica actual de la llamada "gentrificación" de la ciudad de Medellín.

En los últimos años se ha percibido una preocupación general en la ciudad de medellín por los precios cada vez más elevados de vida, muchas personas culpan a los extrangeros que llegan a quedarse en la ciudad y gastan en dolares.

El pensamiento general es que al ellos tener mejor capacidad adquisitiva están encareciendo todo por el cambio de su moneda local (mayormente en dolares)

con este estudio se quieren estudiar las llegadas y salidas internacionales del aeropuerto Jose María Cordoba para ver qué tantos individuos han aumentado la población e intentar predecir cuantas personas van a entrar y salir del país, esto con el objetico de prdecir si la población extrangera tiende a la alza o la baja en el futuro.

en el siguiente paso se cargan los datos de legadas y salidas internacionales desde el años 2007 hasta el año 2022

In [1]:
import pandas as pd
import numpy as np


arrivals = pd.read_csv("./llegada_mensual_pasajeros_aeropuerto_de_origen_internacional.csv", sep = ';')
departures = pd.read_csv("./salida_mensual_pasajeros_aeropuerto_destino_internacional.csv", sep = ';')

arrivals.columns


Index(['lle_periodo', 'lle_origenpax', 'lle_cod', 'lle_indicador',
       'lle_valor'],
      dtype='object')

ahora se usa la fecha de las llegadas y salidas como indice para crear una serie de tiempo

In [2]:
arrivals['lle_periodo'] = pd.to_datetime(arrivals['lle_periodo'], format='%Y%m')

arrivals.set_index('lle_periodo', inplace=True)
arrivals.sort_index(inplace=True)


arrivals.head()


,lle_origenpax,lle_cod,lle_indicador,lle_valor
lle_periodo,,,,
2007-01-01,Colombia,CO,LLEGADA_PAX_INTERNA,164.0
2007-01-01,Canadá,CA,LLEGADA_PAX_INTERNA,7.0
2007-01-01,Venezuela,VE,LLEGADA_PAX_INTERNA,2797.0
2007-01-01,Uruguay,UY,LLEGADA_PAX_INTERNA,18.0
2007-01-01,Turquía,TR,LLEGADA_PAX_INTERNA,0.0


In [3]:
departures['sal_periodo'] = pd.to_datetime(departures['sal_periodo'], format='%Y%m')

departures.set_index('sal_periodo', inplace=True)
departures.sort_index(inplace=True)

departures.head()


,sal_codigo,sal_indicador,sal_valor,sal_destinoint
sal_periodo,,,,
2007-01-01,AN,SALIDA_PAX,906.0,Antillas Neerlandesas
2007-01-01,VE,SALIDA_PAX,4215.0,Venezuela
2007-01-01,UY,SALIDA_PAX,14.0,Uruguay
2007-01-01,US,SALIDA_PAX,16643.0,Estados Unidos
2007-01-01,TT,SALIDA_PAX,36.0,Trinidad y Tobago


en el siguiente paso se hará un análisis de completitud de los datos

In [4]:
arrivals.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5951 entries, 2007-01-01 to 2022-02-01
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   lle_origenpax  5951 non-null   object
 1   lle_cod        5951 non-null   object
 2   lle_indicador  5951 non-null   object
 3   lle_valor      5951 non-null   object
dtypes: object(4)
memory usage: 232.5+ KB


In [5]:
arrivals['lle_origenpax'].unique()

array(['Colombia', 'Canadá', 'Venezuela', 'Uruguay', 'Turquía',
       'Trinidad y Tobago', 'República Dominicana', 'Perú', 'Bolivia',
       'Paraguay', 'Panamá', 'Nicaragua', 'México', 'Costa Rica', 'Haití',
       'Cuba', 'Brasil', 'Ecuador', 'El Salvador', 'España', 'Honduras',
       'Filipinas', 'Guatemala', 'Antillas Neerlandesas', 'Bahamas',
       'Argentina', 'Estados Unidos', 'Chile', 'Jamaica', 'Belice',
       'Guyana', 'Reino Unido', 'Islas Caimán', 'Italia', 'Japón',
       'Portugal', 'China', 'Francia', 'Alemania', 'Australia',
       'Bermudas', 'Barbados', 'Corea del Sur', 'Emiratos Árabes Unidos',
       'Israel', 'Países Bajos', 'Rusia', 'Senegal', 'Suecia', 'Suiza',
       'Bélgica', 'Grecia', 'Qatar', 'Marruecos', 'Guam', 'Irlanda',
       'Otros países del Caribe', 'San Cristóbal y Nieves', 'Sudáfrica',
       'Jordania', 'Puerto Rico', 'Austria', 'Dinamarca', 'Noruega',
       'Nueva Zelanda', 'Hungría', 'Luxemburgo',
       'ISLAS VÍRGENES BRITÁNICAS', 'Costa 

In [6]:
arrivals['lle_valor'] = pd.to_numeric(arrivals['lle_valor'], errors='coerce')
print(arrivals.isnull().sum())


lle_origenpax      0
lle_cod            0
lle_indicador      0
lle_valor        110
dtype: int64


se evidencia que hay datos vacíos en la columna `lle_valor`, se analiza más profundamente para deducir cuál puede ser la causa y con base en eso tratarlos apropiadamente.

se aíslan las filas con valores nulos (NaN) y se valida si podrían ser ceros

In [13]:
null_rows = arrivals[arrivals['lle_valor'].isnull()]
null_rows

,lle_origenpax,lle_cod,lle_indicador,lle_valor
lle_periodo,,,,
2019-04-01,Nueva Zelanda,NZ,LLEGADA_PAX_INTERNA,NaN
2019-04-01,Noruega,NO,LLEGADA_PAX_INTERNA,NaN
2019-04-01,Jordania,JO,LLEGADA_PAX_INTERNA,NaN
2019-04-01,Suecia,SE,LLEGADA_PAX_INTERNA,NaN
2019-04-01,San Cristóbal y Nieves,KN,LLEGADA_PAX_INTERNA,NaN
...,...,...,...,...
2019-12-01,antigua and barbuda,AG,LLEGADA_PAX_INTERNA,NaN
2019-12-01,Barbados,BB,LLEGADA_PAX_INTERNA,NaN
2019-12-01,Belice,BZ,LLEGADA_PAX_INTERNA,NaN


en esta lista de países con `lle_valor` se pueden notar duplicados, ahora se validará si son todos paíese que no es comín que lleguen pasajeros a colombia

In [14]:
print(null_rows['lle_origenpax'].unique())

['Nueva Zelanda' 'Noruega' 'Jordania' 'Suecia' 'San Cristóbal y Nieves'
 'saint lucia' 'Senegal' 'Croacia' 'Dinamarca' 'india' 'Irlanda'
 'Sudáfrica' 'antigua and barbuda' 'Nicaragua' 'Belice'
 'Trinidad y Tobago' 'Jamaica' 'Marruecos' 'Qatar' 'Islas Caimán'
 'Barbados' 'Haití' 'Venezuela']


de lo anterior se puede deducir que no son países muy comunes por lo tanto es muy probable que los valores sean cero. por eso se procederá a reemplazar los datos nulos por ceros

In [15]:
arrivals['lle_valor'].fillna(0, inplace=True)
print(arrivals.isnull().sum())

lle_origenpax    0
lle_cod          0
lle_indicador    0
lle_valor        0
dtype: int64
